In [ ]:
## RUN THIS CELL IF AND ONLY IF THIS IS OPEN IN GOOGLE COLAB
!git clone https://github.com/ries-lab/SimuFLUX.git
%cd SimuFLUX/python/examples

In [ ]:
%matplotlib inline

import sys
import os
from pathlib import Path

SCRIPT_DIR = Path(os.getcwd()).parent
sys.path.append(os.path.dirname(SCRIPT_DIR))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from python.psfs import PsfDonut2D
from python.estimators import est_qLSQiter2D
from python.fluorophores import FlBlinkBleach
from python.simulators import Simulator

In [ ]:
fl = FlBlinkBleach()
sim = Simulator(fluorophores=fl)
sim.fluorophores.fast_toff = 0.1  # off-time ms
sim.fluorophores.fast_ton = 0.1  # on-time ms

In [ ]:
repetitions = 1  # how often to repeat the pattern scan
L = 75
pointdwelltimerep = 0.1  # ms, for all repetitions
pointdwelltime=pointdwelltimerep/repetitions  # ms
orbitpoints = 4
probecenter = True
psf_donut = PsfDonut2D()
sim.definePattern("donut", 
                  psf_donut, 
                  makepattern = "orbitscan", 
                  orbitpoints = orbitpoints,
                  probecenter = probecenter,
                  orbitL = L,
                  pointdwelltime = pointdwelltime,
                  laserpower = 100,
                  repetitions = repetitions)
sim.defineComponent("estsq", "estimator", est_qLSQiter2D, parameters=[L,probecenter], dim=(0,1))
out = sim.runSequence(["donut", "estsq"])
sim.summarize_results(out)

In [ ]:
## plot std vs repetitions
allrepetitions = np.arange(25)+1
stdx = np.zeros((len(allrepetitions),1))
stdy = np.zeros((len(allrepetitions),1))
stdxrel = np.zeros((len(allrepetitions),1))
stdyrel = np.zeros((len(allrepetitions),1))
biasx = np.zeros((len(allrepetitions),1))
biasy = np.zeros((len(allrepetitions),1))

for k in range(len(allrepetitions)):
    pointdwelltime=pointdwelltimerep/allrepetitions[k]  # us
    sim.definePattern("donut4", 
                      psf_donut, 
                      makepattern = "orbitscan", 
                      orbitpoints = orbitpoints,
                      probecenter = probecenter,
                      orbitL = L,
                      pointdwelltime = pointdwelltime,
                      laserpower = 100,
                      repetitions = allrepetitions[k])
    out = sim.runSequence(["donut4", "estsq"], maxlocs=3000)
    bright = out.loc.phot > np.quantile(out.loc.phot,0.1)  # filter out localizations that are too dim, 
                                                           # outliers from fluorophores that are mostly off
    converged = (np.abs(out.loc.xnm)<L) & (np.abs(out.loc.ynm)<L)  # estimator is artificially bound by +/-L. Here we 
                                                                   # remove localizations where the estimator converged to this limit.
    sr=sim.summarize_results(out, filter=bright&converged, display=False)
    stdx[k] = sr.std[0]
    stdy[k] = sr.std[1]
    stdxrel[k] = stdx[k]/sr.sCRB[0]
    stdyrel[k] = stdy[k]/sr.sCRB[1]
    biasx[k] = sr.bias[0]
    biasy[k] = sr.bias[1]


In [ ]:
plt.figure()
plt.plot(allrepetitions,stdxrel,allrepetitions,stdyrel,allrepetitions,allrepetitions*0+1,'k')
plt.xlabel('repetitions')
plt.ylabel('std/sCRB')
plt.title("flickering fluorophores, mitigation by repetitive scanning")